In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    max as F_max, col, lit, to_date, year, month, dayofmonth, concat_ws,
    when, row_number, rank, current_timestamp
)
from pyspark.sql.window import Window

# Define the function for fact_course_history creation
def fact_course_history(catalog_name):
    # Initialize SparkSession (already available in Databricks)
    spark = SparkSession.builder.getOrCreate()

    # Load the required tables
    c_path = f"{catalog_name}.schema.fact_course_progress"
    d_path = f"{catalog_name}.schema.dim_fy_completed"
    b_path = f"{catalog_name}.schema.dim_fy_began"
    dc_path = f"{catalog_name}.schema.dim_course_progress"
    c = spark.read.table(c_path)
    d = spark.read.table(d_path)
    b = spark.read.table(b_path)
    dc = spark.read.table(dc_path)

    # Filter the 'dim_fy_completed' table based on the current timestamp
    d_filtered = d.filter(
        (col("fy_start_date") < current_timestamp()) &
        (col("fy_start_date") != to_date(lit("1900-01-01")))
    )

    # Perform joins with the appropriate filters to avoid ambiguity
    joined_df = (
        c.alias("c")
        .join(d_filtered.alias("d"), 
            (col("c.date_began") <= col("d.fy_end_date")) &
            ((col("c.date_completed") == to_date(lit("1900-01-01"))) |
             (col("c.date_completed") >= col("d.fy_start_date"))))
        .join(b.alias("b"),
            (col("c.date_began") <= col("b.fy_end_date")) &
            (col("c.date_began") >= col("b.fy_start_date")))
        .join(dc.alias("dc"), col("c.course_user_id") == col("dc.course_user_id"), how="left")
    )

    # Perform aggregation and select the required columns, renaming them to avoid ambiguity
    aggregated_df = (
        joined_df
        .groupBy("c.course_user_id", "d.fy_completed", "d.fy_end_date")
        .agg(
            F_max("c.cert_id").alias("cert_id"),
            F_max("c.course_id").alias("course_id"),
            F_max("c.course_user_id").alias("c_course_user_id"),  # Renamed to avoid conflict
            F_max("c.date_began").alias("date_began"),
            F_max("c.date_completed").alias("date_completed"),
            F_max("c.program_id").alias("program_id"),
            F_max("c.program_user_id").alias("program_user_id"),
            F_max("c.user_id").alias("user_id"),
            F_max("c.scorm_id").alias("scorm_id"),
            F_max("b.fy_began").alias("fy_began"),
            F_max("dc.time_started").alias("time_started"),
            F_max("dc.completion_status").alias("completion_status"),
            F_max("dc.course_user_id").alias("dc_course_user_id")  # Renamed to avoid conflict
        )
        .withColumn("certification_user_id", lit(-1).cast("bigint"))
        .withColumn("event_id", lit(-1))
        .withColumn("session_id", lit(-1))
        .withColumn("fy_status", col("d.fy_completed").cast("int"))
    )

    # Calculate the course completion status based on conditions
    aggregated_df = aggregated_df.withColumn(
        "coursecompletionstatusforfy",
        when(col("completion_status") == "Not yet started", "Not yet started")
        .when(col("d.fy_completed") == col("d.fy_completed"), "Complete")
        .when(
            to_date(concat_ws("-", year("time_started"), month("time_started"), dayofmonth("time_started")))
            > col("d.fy_end_date"), "Not yet started")
        .when(col("completion_status") == "In progress", "In progress")
        .when(year("date_completed") == 1900, "Not yet started")
        .when(col("date_completed") <= col("d.fy_end_date"), "Complete")
        .otherwise("In progress")
    )

    # Add a limittofactcoursehistory column with a constant value of 1
    aggregated_df = aggregated_df.withColumn("limittofactcoursehistory", lit(1))

    # Define the window specification for ranking and row numbering
    window_spec_rank = Window.orderBy(col("c_course_user_id"), col("d.fy_completed"))
    window_spec_row = Window.partitionBy(col("c_course_user_id")).orderBy(col("d.fy_completed"))

    # Add windowed rank and row number columns
    fact_course_history_df = aggregated_df \
        .withColumn("uniquecoursehistoryrecord", rank().over(window_spec_rank)) \
        .withColumn("historyrecordno", row_number().over(window_spec_row))

    # Save the result as a Delta table
    output_path = f"{catalog_name}.schema.fact_course_history"
    fact_course_history_df.write.format("delta").mode("overwrite").saveAsTable(output_path)

    return fact_course_history_df
